# CoppeliaSim: Python interface
### Setup
Make sure you have following files in your directory, in order to run the various examples:

1. sim.py and simConst.py in coppelia's folder ```programming/remoteApiBindings/python/python
2. the appropriate remote API library: "remoteApi.dll" (Windows), "remoteApi.dylib" (Mac) or "remoteApi.so" (Linux). In coppelia's folder ```programming/remoteApiBindings/lib/lib/


### Before executing



* Make sure to have the server side running in CoppeliaSim.

* In a child script of a CoppeliaSim scene, add following command to be executed just once, at simulation start:
```python
simRemoteApi.start(19999)
```

* then start simulation, and run this program.

* For each successful call to simxStart, there should be a corresponding call to simxFinish at the end!


### Tutorials
* Python tutorial: http://34.208.13.223/VREP/04PythonTutorial/
* CoppeliaSim tutorial: https://www.youtube.com/playlist?list=PLjzuoBhdtaXOoqkJUqhYQletLLnJP8vjZ
* API Documentation: https://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm

# Scene
For testing purposes, you can use the file of the class: three_robots.ttt.
However, I recommend to start with an empty scene and add the robots.


In [2]:
# Import the python library that connects to CoppeliaSim, the file is sim.py.
try:
    import sim
except:
    print('--------------------------------------------------------------')
    print('"sim.py" could not be imported. This means very probably that')
    print('either "sim.py" or the remoteApi library could not be found.')
    print('Make sure both are in the same folder as this file,')
    print('or appropriately adjust the file "sim.py"')
    print('--------------------------------------------------------------')
    print('')

import time

%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Check if python is connecting to Coppelia

In [3]:
print('Program started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    sim.simxGetPingTime(clientID)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
else:
    print('Failed connecting to remote API server')
print('Program ended')

Program started
Connected to remote API server
Program ended


# Controlling motors

1. Add a Pioneer_p3dx robot.
2. Remove its Lua script.
3. Start the remote API server
4. Send a control command with the following script

In [12]:
# Actuator names
left_motor_name = "Pioneer_p3dx_leftMotor"
right_motor_name = "Pioneer_p3dx_rightMotor"

# Values to send  (rad/sec)
left_velocity = -1.0
right_velocity = 1.0


# Send the command!
print('Program started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)
    
    # Get handlers or actuators
    err_code, l_motor_handle = sim.simxGetObjectHandle(clientID, left_motor_name, sim.simx_opmode_blocking)
    err_code, r_motor_handle = sim.simxGetObjectHandle(clientID, right_motor_name, sim.simx_opmode_blocking)

    # Send the values!
    err_code = sim.simxSetJointTargetVelocity(clientID, l_motor_handle, left_velocity, sim.simx_opmode_streaming)
    err_code = sim.simxSetJointTargetVelocity(clientID, r_motor_handle, right_velocity, sim.simx_opmode_streaming)

    # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    sim.simxGetPingTime(clientID)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
else:
    print('Failed connecting to remote API server')
print('Program ended')

Program started
Connected to remote API server
Program ended


# Object oriented programming

In [4]:
class robot():
    
    def __init__(self, motor_names, frame_name=None):        
        self.client_id = self.open_connection()
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  None
        if frame_name is not None:
            self.frame = self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object):
        relative_handle = self._get_handler(relative_object)
        sim.simxSetObjectPosition(clientID, self.frame, relative_handle, position, sim.simx_opmode_oneshot)

In [ ]:
# Joint names in coppelia for KUKA omnidirectional robot
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']


r = robot(motor_names)  # Create an instance of our robot
# r.send_motor_velocities([1., -1., -1., 1.])  # Move the robot

r.close_connection()  # End

# Control loop

In [168]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot(motor_names)  # Create an instance of our robot

time_steps = linspace(0, 230, 2300)

for t in time_steps:
    if r.client_id == -1:  # Check connection
        print('Robot is not connected')
        break
    elif t < 6.6300:
        r.send_motor_velocities([1.00, 1.00, 1.00, 1.00])
    # hit red1
    elif t < 30.0000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 37.2500:
        r.send_motor_velocities([-1.00, -1.00, -1.00, -1.00])
    # hit red2
    elif t < 49.0000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 56.9000:
        r.send_motor_velocities([-1.00, -1.00, -1.00, -1.00])
    # hit red3
    elif t < 73.0000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 78.6300:
        r.send_motor_velocities([1.00, 1.00, 1.00, 1.00])
    elif t < 88.5000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 96.5000:
        r.send_motor_velocities([1.00, 1.00, 1.00, 1.00])
    # hit red4 here
    elif t < 113.0000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    # turn into corner
    elif t < 120.5000:
        r.send_motor_velocities([-1.00, -1.00, -1.00, -1.00])
    elif t < 126.5000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    # turn towards red5
    elif t < 132.6000:
        r.send_motor_velocities([-1.00, -1.00, -1.00, -1.00])
    # should hit red5 here
    elif t < 143.5000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    # turn towards edge
    elif t < 158.5000:
        r.send_motor_velocities([1.00, 1.00, 1.00, 1.00])
    # hail mary pass
    elif t < 169.5000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 176.5000:
        r.send_motor_velocities([1.00, 1.00, 1.00, 1.00])
    # move back to middle
    elif t < 183.0000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 190.0000:
        r.send_motor_velocities([-1.00, -1.00, -1.00, -1.00])
    elif t < 198.0000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 204.0000:
        r.send_motor_velocities([-1.00, -1.00, -1.00, -1.00])
    # go up decently far
    elif t < 211.5000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    elif t < 217.2500:
        r.send_motor_velocities([1.00, 1.00, 1.00, 1.00])
    # hit the sixth one
    elif t < 229.0000:
        vx, vy = 5.00, 0
        r.send_motor_velocities([vx - vy, -vx - vy, -vx + vy, vx + vy])
    else:
        r.send_motor_velocities([0., 0., 0., 0.])
    time.sleep(0.1)

    
r.send_motor_velocities([0, 0, 0, 0])

r.close_connection()  # End

Robot connected
Connection closed


# Drone control

In [ ]:
r = robot([], frame_name='Quadricopter_target')
r.set_position([5.875,7.89,0.48], 'Cuboid')
r.close_connection()

### Lets make the drone to follow a circle
* Add a primitive shape -> cuboid with the name 'Cuboid'. Then the drone can move around it in circles

In [75]:
r = robot([], frame_name='Quadricopter_target')

time_steps = linspace(0, 90, 900)



for t in time_steps:
    if r.client_id == -1:  # Check connection
        print('Robot is not connected')
        break
    elif t <= 22:
        px, py, pz = 5.875+-0.2*t+0.015960743801652896*t**2+-0.0006902704733283244*t**3, 8.05+-0.5*t+0.0059400826446280915*t**2+0.0001643501126972203*t**3, 0.51+0.02*t+0.001890495867768596*t**2+-3.662659654395193e-05*t**3
        r.set_position([px,py,pz], 'Cuboid')
    elif t <= 35:
        px, py, pz = 1.85+-0.5*(t-22)+-0.022928994082840236*(t-22)**2+0.002162039144287665*(t-22)**3, 2.675+0.0*(t-22)+0.08801775147928995*(t-22)**2+-0.006486117432862995*(t-22)**3, 1.475+0.05*(t-22)+-0.037721893491124266*(t-22)**2+0.002822030040964953*(t-22)**3
        r.set_position([px,py,pz], 'Cuboid')
    elif t <= 49:
        px, py, pz = -3.775+0.0*t+0.0*t**2+0.0*t**3, 3.3+-1.0*(t-35)+0.0*(t-35)**2+0.005102040816326531*(t-35)**3, 2.2+0.5*(t-35)+-0.05089285714285714*(t-35)**2+0.0024234693877551016*(t-35)**3
        r.set_position([px,py,pz], 'Cuboid')
    elif t <= 60:
        px, py, pz = -3.775+0.0*(t-49)+0.08471074380165289*(t-49)**2+0.0003756574004507889*(t-49)**3, 2.5+2.0*(t-49)+-0.32479338842975203*(t-49)**2+0.011419984973703983*(t-49)**3, 5.625+0.5*(t-49)+-0.193801652892562*(t-49)**2+0.010368144252441773*(t-49)**3
        r.set_position([px,py,pz], 'Cuboid')
    elif t <= 72:
        px, py, pz = 6.975+2.0*(t-60)+-0.21562499999999998*(t-60)**2+0.002719907407407407*(t-60)**3, 0.5+-1.0*(t-60)+0.04062500000000001*(t-60)**2+-0.0010995370370370373*(t-60)**3, 1.475+0.0*t+0.0*t**20.0*t**3
        r.set_position([px,py,pz], 'Cuboid')
    elif t <= 83:
        px, py, pz = 4.625+-1.0*(t-72)+0.022727272727272728*(t-72)**2+-0.004132231404958678*(t-72)**3, -7.85+-2.0*(t-72)+0.30785123966942146*(t-72)**2+-0.012321562734785874*(t-72)**3, 1.475+0.0*t+0.0*t**2+0.0*t**3
        r.set_position([px,py,pz], 'Cuboid')
    else:
        time.sleep(0.01)
    
    time.sleep(0.02)

    


r.close_connection()  # End

Robot connected
Connection closed
